In [10]:
import pandas as pd
from datetime import datetime

In [11]:
# Đọc file CSV
df = pd.read_csv("all_invoice_info.csv", quotechar='"', sep=",", encoding="utf-8")

In [12]:
def convert_to_datetime(date_str):
    try:
        if date_str and isinstance(date_str, str):  # Kiểm tra nếu date_str không rỗng và là chuỗi
            # Loại bỏ 3 ký tự cuối nếu chuỗi chứa "AM" hoặc "PM"
            cleaned_date_str = date_str[:-3].strip()
            # Chuyển đổi chuỗi thành datetime
            return datetime.strptime(cleaned_date_str, '%d/%m/%Y %H:%M:%S')  # Định dạng dd/mm/yyyy hh:mm:ss
        return None  # Trả về None nếu không có chuỗi hợp lệ
    except ValueError:
        return None  # Trả về None nếu không thể chuyển đổi

In [13]:
df['created_at'] = df['created_at'].apply(convert_to_datetime)
df['last_modified_at'] = df['last_modified_at'].apply(convert_to_datetime)

In [15]:
import ast

columns_to_convert = ['co_info']
for col in columns_to_convert:
    if col in df.columns:  # Kiểm tra nếu cột tồn tại
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)

In [16]:
df = df.fillna('')

In [19]:
import pyodbc

# Kết nối tới SQL Server
server = 'localhost,1434'
username = 'sa'
password = '1234QWER@'


In [23]:

# Kết nối đến database master để tạo database mới
connection = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE=master;UID={username};PWD={password}')
cursor = connection.cursor()

# Tạo database nếu chưa tồn tại
try:
    cursor.execute("IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = 'COData') "
                   "BEGIN "
                   "CREATE DATABASE COData; "
                   "END")
    connection.commit()  # Commit lệnh CREATE DATABASE
except Exception as e:
    print("Lỗi khi tạo cơ sở dữ liệu:", e)

# Đóng kết nối và mở lại kết nối tới database mới
connection.close()

# Kết nối đến database mới
connection = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE=COData;UID={username};PWD={password}')
cursor = connection.cursor()

# Tạo bảng C/O
cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'invoice')
    BEGIN
        CREATE TABLE invoice (
            invoice_doc_id VARCHAR(50) PRIMARY KEY,
            status NVARCHAR(50),
            ordercode VARCHAR(20),
            companyname NVARCHAR(100),
            companytaxcode VARCHAR(50),
            companyaddress NTEXT,
            companyemail VARCHAR(100),
            amount VARCHAR(20),
            invoice_receip_no VARCHAR(50),
            invoice_address_label VARCHAR(50),
            another_email NVARCHAR(50),
            created_at DATETIME,
            last_modified_at DATETIME
        );
    END
""")
#tạo bảng transport
cursor.execute("""

    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'invoice_co_info')
    BEGIN
        CREATE TABLE invoice_co_info (
            invoice_doc_id VARCHAR(50),
            item_num smallint,
            co_num VARCHAR(50),
            fee_type NVARCHAR(50),
            fee NVARCHAR(50),
            CONSTRAINT FK_invoice_co_info FOREIGN KEY (invoice_doc_id) REFERENCES invoice(invoice_doc_id)
        );
    END                       
""")

connection.commit()  # Commit các lệnh tạo bảng

# Đóng kết nối
cursor.close()
connection.close()

print("Cơ sở dữ liệu và các bảng đã được tạo thành công.")


Cơ sở dữ liệu và các bảng đã được tạo thành công.


In [27]:
# Kết nối đến SQL Server
connection = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE=COData;UID={username};PWD={password}')
cursor = connection.cursor()

# Chèn dữ liệu vào bảng invoice
for idx,record in df.iterrows():
    cursor.execute("""
        INSERT INTO invoice (invoice_doc_id,status,ordercode,companyname,companytaxcode,companyaddress,companyemail,amount,invoice_receip_no,invoice_address_label,another_email,created_at,last_modified_at)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, record['doc_id'], record['status'], record['ordercode'], 
          record['companyname'], record['companytaxcode'], record['companyaddress'],record['companyemail'],
          record['amount'], record['invoice_receip_no'], record['invoice_address_label'],
          record['another_email'], str(record['created_at']), str(record['last_modified_at'])
        )

    # Chèn dữ liệu vào bảng invoice_co_info
    for idx, goods in enumerate(record['co_info'][1:], 1):  # Bỏ qua dòng tiêu đề
        cursor.execute("""
            INSERT INTO invoice_co_info (invoice_doc_id,item_num, co_num, fee_type, fee)
            VALUES (?, ?, ?, ?, ?)
        """, record['doc_id'], idx, goods[0], goods[2], goods[3])


# Commit các thay đổi vào cơ sở dữ liệu
connection.commit()

# Đóng kết nối
cursor.close()
connection.close()

print("Dữ liệu đã được chèn thành công.")


Dữ liệu đã được chèn thành công.
